In [1]:
import pandas as pd
import numpy as np

In [2]:
descr_df = pd.read_csv('data_dictionary.csv')

|index|Field|Possible Values|Comments|
|---|---|---|---|
|0|fsa\_id|int|Food Standard Agency's ID for this pub\.|
|1|name|string|Name of the pub\.|
|2|address|string|Address fields separated by commas\.|
|3|postcode|string|Postcode of the pub\.|
|4|easting|int|NaN|
|5|northing|int|NaN|
|6|latitude|decimal|NaN|
|7|longitude|decimal|NaN|
|8|local\_authority|string|Local authority this pub falls under\.|

In [3]:
pubs = pd.read_csv('open_pubs.csv', names=descr_df['Field'])
pubs.head()

,fsa_id,name,address,postcode,easting,northing,latitude,longitude,local_authority
0,22,Anchor Inn,"Upper Street, Stratford St Mary, COLCHESTER",CO7 6LW,604749,234404,51.970379,0.979340,Babergh
1,36,Ark Bar Restaurant,"Ark Bar And Restaurant, Cattawade Street, Bran...",CO11 1RH,610194,233329,51.958698,1.057832,Babergh
2,74,Black Boy,"The Lady Elizabeth, 7 Market Hill, SUDBURY, Su...",CO10 2EA,587334,241316,52.038595,0.729915,Babergh
3,75,Black Horse,"Lower Street, Stratford St Mary, COLCHESTER",CO7 6JS,622675,-5527598,\N,\N,Babergh
4,76,Black Lion,"Lion Road, Glemsford, SUDBURY",CO10 7RF,622675,-5527598,\N,\N,Babergh


In [4]:
pubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51331 entries, 0 to 51330
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fsa_id           51331 non-null  int64 
 1   name             51331 non-null  object
 2   address          51331 non-null  object
 3   postcode         51331 non-null  object
 4   easting          51331 non-null  int64 
 5   northing         51331 non-null  int64 
 6   latitude         51331 non-null  object
 7   longitude        51331 non-null  object
 8   local_authority  51331 non-null  object
dtypes: int64(3), object(6)
memory usage: 3.5+ MB


In [5]:
pubs = pubs.drop(pubs[pubs['latitude'] == '\\N'].index)
pubs['longitude'] = pubs['longitude'].astype('float')
pubs['latitude'] = pubs['latitude'].astype('float')
pubs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50564 entries, 0 to 51330
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fsa_id           50564 non-null  int64  
 1   name             50564 non-null  object 
 2   address          50564 non-null  object 
 3   postcode         50564 non-null  object 
 4   easting          50564 non-null  int64  
 5   northing         50564 non-null  int64  
 6   latitude         50564 non-null  float64
 7   longitude        50564 non-null  float64
 8   local_authority  50564 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 3.9+ MB


In [6]:
pubs = pubs.drop_duplicates(subset=['latitude',	'longitude'])

In [7]:
import requests
from bs4 import BeautifulSoup

areasurl="https://ideal-postcodes.co.uk/guides/postcode-areas"
response=requests.get(areasurl)
soup = BeautifulSoup(response.text, 'html.parser')
areas_table = soup.find('table')
areas = pd.read_html(str(areas_table))
areas = pd.DataFrame(areas[0])
areas.head(10)

,Postcode Area,Postcode Area Name,Region
0,AB,Aberdeen,Scotland
1,AL,St. Albans,East of England
2,B,Birmingham,West Midlands
3,BA,Bath,South West
4,BB,Blackburn,North West
5,BD,Bradford,North West
6,BH,Bournemouth,South West
7,BL,Bolton,North West
8,BN,Brighton,South East
9,BR,Bromley,Greater London


In [8]:
pubs['postcode'].sample(20).values

array(['BS2 8DE', 'WV1 1DN', 'WF4 3DS', 'TN14 7TJ', 'WA9 1AF', 'M9 7ED',
       'B24 8QJ', 'IP2 9BA', 'NG19 6SG', 'NE9 7JR', 'E2 7NS', 'RG40 1AP',
       'DA8 3HB', 'EX15 2NT', 'CM22 7HF', 'EH41 4DL', 'NN10 0SA',
       'SA2 9BH', 'SN13 0HF', 'S60 2NF'], dtype=object)

In [9]:
import re

regex = r'^[A-Z]{1,2}'
pubs['Postcode Area'] = pubs['postcode'].apply(lambda x: re.search(regex, x).group(0))
pubs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47338 entries, 0 to 51330
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fsa_id           47338 non-null  int64  
 1   name             47338 non-null  object 
 2   address          47338 non-null  object 
 3   postcode         47338 non-null  object 
 4   easting          47338 non-null  int64  
 5   northing         47338 non-null  int64  
 6   latitude         47338 non-null  float64
 7   longitude        47338 non-null  float64
 8   local_authority  47338 non-null  object 
 9   Postcode Area    47338 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 4.0+ MB


In [10]:
pubs = pubs.merge(areas[['Postcode Area', 'Region']], how='left', on='Postcode Area')
pubs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47338 entries, 0 to 47337
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fsa_id           47338 non-null  int64  
 1   name             47338 non-null  object 
 2   address          47338 non-null  object 
 3   postcode         47338 non-null  object 
 4   easting          47338 non-null  int64  
 5   northing         47338 non-null  int64  
 6   latitude         47338 non-null  float64
 7   longitude        47338 non-null  float64
 8   local_authority  47338 non-null  object 
 9   Postcode Area    47338 non-null  object 
 10  Region           47338 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 4.3+ MB


In [11]:
pubs['Region'].value_counts()

North West         6878
South East         5808
Greater London     4857
South West         4804
North East         4763
West Midlands      4755
East Midlands      4479
East of England    3512
Scotland           3385
Wales              3332
East England        765
Name: Region, dtype: int64

In [12]:
pubs['Region'] =  pubs['Region'].replace('East England', 'East of England')

In [13]:
pubs_new = pubs[['name', 'address', 'latitude', 'longitude', 'Region']]
pubs_new.to_csv('pubs.csv', index=False)